In [1]:
# train_models_robust.py
import pandas as pd
import joblib
import inspect
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, 
    AdaBoostClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# ----------------------------
# 1. Load data
# ----------------------------
df = pd.read_csv("small.csv")   # change path if needed

# assume last column is target
X = df.iloc[:, :-1].copy()
y = df.iloc[:, -1].copy()

# ----------------------------
# 2. Label-encode target
# ----------------------------
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# ----------------------------
# 3. Prepare OneHotEncoder in a version-safe way
# ----------------------------
# determine supported parameter name for OneHotEncoder dense output
ohe_init_sig = inspect.signature(OneHotEncoder.__init__)
ohe_kwargs = {"handle_unknown": "ignore"}
if "sparse_output" in ohe_init_sig.parameters:
    ohe_kwargs["sparse_output"] = False
elif "sparse" in ohe_init_sig.parameters:
    ohe_kwargs["sparse"] = False
else:
    # fallback: do not pass either (should be rare)
    pass

ohe = OneHotEncoder(**ohe_kwargs)

# ----------------------------
# 4. Identify categorical and numeric columns
# ----------------------------
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object", "category"]).columns.tolist()

# Fit a standalone scaler on numeric columns (so we can save scaler.pkl)
if len(num_cols) > 0:
    numeric_scaler_fitted = StandardScaler().fit(X[num_cols])
else:
    numeric_scaler_fitted = None

# For pipeline we provide a fresh transformer instance (it will be fitted inside pipeline.fit)
numeric_scaler_for_pipeline = StandardScaler()

# Build ColumnTransformer
transformers = []
if len(cat_cols) > 0:
    transformers.append(("cat", ohe, cat_cols))
if len(num_cols) > 0:
    transformers.append(("num", numeric_scaler_for_pipeline, num_cols))

from sklearn.compose import ColumnTransformer
preprocess = ColumnTransformer(transformers=transformers, remainder="drop")

# ----------------------------
# 5. Define models to train
# ----------------------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_jobs=-1),
    "HistGradientBoosting": HistGradientBoostingClassifier(),
    "Extra Trees": ExtraTreesClassifier(n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(probability=True)
}

results = {}

# ----------------------------
# 6. Train & evaluate
# ----------------------------
# we'll use a single train/test split for quick comparisons
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X, y_encoded, test_size=0.20, random_state=42, stratify=y_encoded
)

for name, estimator in models.items():
    pipe = Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", estimator)
    ])
    try:
        pipe.fit(X_train_full, y_train_full)
        preds = pipe.predict(X_test_full)
        acc = accuracy_score(y_test_full, preds)
        f1 = f1_score(y_test_full, preds, average="weighted")
        results[name] = {"accuracy": acc, "f1": f1}
        print(f"{name} -> acc: {acc:.4f}, f1 (weighted): {f1:.4f}")
    except Exception as e:
        # If a model can't train for some dataset shapes, record the error and continue
        results[name] = {"error": str(e)}
        print(f"{name} -> ERROR: {e}")

# ----------------------------
# 7. Choose best model (by weighted F1)
# ----------------------------
# filter out models that errored
valid_results = {k: v for k, v in results.items() if "f1" in v}
if not valid_results:
    raise RuntimeError("No model trained successfully. See printed errors above.")

best_name = max(valid_results, key=lambda k: valid_results[k]["f1"])
print("\nBest model selected:", best_name, valid_results[best_name])

best_estimator = models[best_name]

# ----------------------------
# 8. Retrain best model on full dataset
# ----------------------------
final_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", best_estimator)
])

final_pipeline.fit(X, y_encoded)  # fit on entire data

# ----------------------------
# 9. Save artifacts
# ----------------------------
joblib.dump(final_pipeline, "model.pkl")
# if numeric_scaler_fitted is None, save None to indicate not applicable
joblib.dump(numeric_scaler_fitted, "scaler.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("\nSaved: model.pkl, scaler.pkl, label_encoder.pkl")


Logistic Regression -> ERROR: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
Random Forest -> acc: 0.8025, f1 (weighted): 0.8000
HistGradientBoosting -> acc: 0.8000, f1 (weighted): 0.7976
Extra Trees -> acc: 0.7275, f1 (weighted): 0.7230
Gradient Boosting -> ERROR: Input X contains NaN.
GradientBoostingClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might 

In [3]:
# retrain_fixed.py
"""
Robust training script that:
 - Drops identifier/time columns (vm_id, timestamp) by default
 - Imputes missing values for numeric/categorical features
 - Uses OneHotEncoder in a version-safe way
 - Avoids multiprocessing (no n_jobs=-1 or parallel CV)
 - Falls back to single-fit evaluation if CV fails
 - Saves: model.pkl, scaler.pkl, label_encoder.pkl
"""
import pandas as pd
import joblib
import inspect
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# ----------------------------
# 1. Load data
# ----------------------------
df = pd.read_csv("small.csv")   # adjust path if needed
print("Data shape:", df.shape)

# ----------------------------
# 2. Drop identifier/timecolumns (common in VM logs)
# ----------------------------
drop_cols = []
for c in ["vm_id", "timestamp"]:
    if c in df.columns:
        drop_cols.append(c)
if drop_cols:
    print("Dropping identifier/time columns:", drop_cols)
    df = df.drop(columns=drop_cols)

# ----------------------------
# 3. Prepare X, y
# ----------------------------
# assume last column is the target (as before)
X = df.iloc[:, :-1].copy()
y = df.iloc[:, -1].copy()

print("\nMissing per column:\n", X.isna().sum())
print("\nTarget distribution:\n", y.value_counts())

# ----------------------------
# 4. Label encode target
# ----------------------------
label_enc = LabelEncoder()
y_enc = label_enc.fit_transform(y)

# ----------------------------
# 5. Identify categorical and numeric columns
# ----------------------------
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print("\nCategorical cols:", cat_cols)
print("Numeric cols:", num_cols)

# ----------------------------
# 6. Safe OneHotEncoder for different sklearn versions
# ----------------------------
ohe_init_sig = inspect.signature(OneHotEncoder.__init__)
ohe_kwargs = {"handle_unknown": "ignore"}
if "sparse_output" in ohe_init_sig.parameters:
    ohe_kwargs["sparse_output"] = False
elif "sparse" in ohe_init_sig.parameters:
    ohe_kwargs["sparse"] = False

ohe = OneHotEncoder(**ohe_kwargs)

# ----------------------------
# 7. Imputers & scalers
# ----------------------------
num_imputer = SimpleImputer(strategy="median")     # robust to outliers
cat_imputer = SimpleImputer(strategy="most_frequent", fill_value="__MISSING__")
scaler_for_pipeline = StandardScaler()

# Fit a standalone scaler for numeric columns (to save scaler.pkl)
if len(num_cols) > 0:
    # fit on full numeric data after imputing median to avoid NaNs
    num_data = X[num_cols].copy()
    num_data_imputed = num_imputer.fit_transform(num_data)
    fitted_scaler = StandardScaler().fit(num_data_imputed)
else:
    fitted_scaler = None

# For pipeline preprocessor create fresh instances (they will be fitted inside the pipeline)
num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]) if num_cols else "drop"
cat_pipeline = Pipeline([("imputer", SimpleImputer(strategy="most_frequent", fill_value="__MISSING__")), ("ohe", ohe)]) if cat_cols else "drop"

transformers = []
if cat_cols:
    transformers.append(("cat", cat_pipeline, cat_cols))
if num_cols:
    transformers.append(("num", num_pipeline, num_cols))

preprocessor = ColumnTransformer(transformers=transformers, remainder="drop")

# ----------------------------
# 8. Models (no heavy parallelism)
# ----------------------------
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    # avoid n_jobs=-1 to prevent multiprocessing issues in constrained envs
    "RandomForest": RandomForestClassifier(n_jobs=1, random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_jobs=1, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(probability=True)
}

# ----------------------------
# 9. Train-test split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.20, random_state=42, stratify=y_enc
)

# ----------------------------
# 10. Evaluate with CV (safe: n_jobs=1) with fallback handling
# ----------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}
for name, estimator in models.items():
    pipe = Pipeline([("preprocessor", preprocessor), ("model", estimator)])
    print(f"\nEvaluating {name} ...")
    try:
        # Use cross_val_score with n_jobs=1 to avoid multiprocessing issues
        scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="f1_weighted", n_jobs=1)
        results[name] = {"cv_f1_mean": float(scores.mean()), "cv_f1_std": float(scores.std())}
        print(f"  CV F1 (weighted): {scores.mean():.4f} +/- {scores.std():.4f}")
    except Exception as e:
        # If CV fails, attempt a simple fit on train and evaluate on test (safe single-run)
        print(f"  CV failed for {name}: {e}")
        try:
            pipe.fit(X_train, y_train)
            preds = pipe.predict(X_test)
            acc = accuracy_score(y_test, preds)
            f1 = f1_score(y_test, preds, average="weighted")
            results[name] = {"cv_f1_mean": float(f1), "cv_f1_std": 0.0, "note": "fallback_single_eval"}
            print(f"  Fallback evaluation -> acc: {acc:.4f}, f1 (weighted): {f1:.4f}")
        except Exception as e2:
            results[name] = {"error": str(e2)}
            print(f"  Fallback also failed for {name}: {e2}")

# ----------------------------
# 11. Select best model
# ----------------------------
valid = {k: v for k, v in results.items() if "cv_f1_mean" in v}
if not valid:
    raise RuntimeError("No model evaluated successfully. See printed errors above.")

best_name = max(valid, key=lambda k: valid[k]["cv_f1_mean"])
print("\nBest model selected:", best_name, valid[best_name])
best_estimator = models[best_name]

# ----------------------------
# 12. Retrain best model on FULL DATA
# ----------------------------
final_pipeline = Pipeline([("preprocessor", preprocessor), ("model", best_estimator)])
print("\nFitting final pipeline on full dataset ...")
final_pipeline.fit(X, y_enc)

# ----------------------------
# 13. Evaluate final model on holdout (X_test)
# ----------------------------
try:
    preds_test = final_pipeline.predict(X_test)
    print("\nFinal model test accuracy:", accuracy_score(y_test, preds_test))
    print("Final model test F1 (weighted):", f1_score(y_test, preds_test, average="weighted"))
    print("Classification Report:\n", classification_report(y_test, preds_test, target_names=label_enc.classes_))
except Exception as e:
    print("Could not evaluate final model on X_test:", e)

# ----------------------------
# 14. Save artifacts
# ----------------------------
joblib.dump(final_pipeline, "model.pkl")
joblib.dump(fitted_scaler, "scaler.pkl")
joblib.dump(label_enc, "label_encoder.pkl")
print("\nSaved artifacts: model.pkl, scaler.pkl, label_encoder.pkl")


Data shape: (1998, 13)
Dropping identifier/time columns: ['vm_id', 'timestamp']

Missing per column:
 cpu_usage                    260
memory_usage                 190
network_traffic              208
power_consumption            209
num_executed_instructions    200
execution_time               228
energy_efficiency            142
task_type                    179
task_priority                203
task_status                  190
dtype: int64

Target distribution:
 Anomaly status
1    999
0    999
Name: count, dtype: int64

Categorical cols: ['task_type', 'task_priority', 'task_status']
Numeric cols: ['cpu_usage', 'memory_usage', 'network_traffic', 'power_consumption', 'num_executed_instructions', 'execution_time', 'energy_efficiency']

Evaluating LogisticRegression ...
  CV F1 (weighted): 0.7093 +/- 0.0196

Evaluating RandomForest ...
  CV F1 (weighted): 0.7717 +/- 0.0147

Evaluating HistGradientBoosting ...
  CV F1 (weighted): 0.7801 +/- 0.0089

Evaluating ExtraTrees ...
  CV F1 (weigh

In [4]:
# ----------------------------
# 13B. Confusion Matrix for each model
# ----------------------------
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

print("\nGenerating confusion matrices for all models...\n")

for name, estimator in models.items():
    try:
        pipe = Pipeline([("preprocessor", preprocessor), ("model", estimator)])
        pipe.fit(X_train, y_train)

        preds = pipe.predict(X_test)

        cm = confusion_matrix(y_test, preds)
        print(f"\n{name} Confusion Matrix:\n{cm}\n")

        # Plot confusion matrix
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_enc.classes_)
        fig, ax = plt.subplots(figsize=(4, 4))
        disp.plot(ax=ax, cmap="Blues", colorbar=False)
        plt.title(f"Confusion Matrix - {name}")

        # Save confusion matrix PNG
        fig.savefig(f"cm_{name}.png", dpi=200, bbox_inches="tight")
        plt.close(fig)

        print(f"Saved: cm_{name}.png")

    except Exception as e:
        print(f"❌ Could not generate confusion matrix for {name}: {e}")



Generating confusion matrices for all models...


LogisticRegression Confusion Matrix:
[[115  85]
 [ 37 163]]

Saved: cm_LogisticRegression.png

RandomForest Confusion Matrix:
[[149  51]
 [ 29 171]]

Saved: cm_RandomForest.png

HistGradientBoosting Confusion Matrix:
[[137  63]
 [ 27 173]]

Saved: cm_HistGradientBoosting.png

ExtraTrees Confusion Matrix:
[[127  73]
 [ 49 151]]

Saved: cm_ExtraTrees.png

GradientBoosting Confusion Matrix:
[[145  55]
 [ 29 171]]

Saved: cm_GradientBoosting.png

AdaBoost Confusion Matrix:
[[155  45]
 [ 44 156]]

Saved: cm_AdaBoost.png

KNN Confusion Matrix:
[[109  91]
 [ 65 135]]

Saved: cm_KNN.png

SVC Confusion Matrix:
[[118  82]
 [ 37 163]]

Saved: cm_SVC.png


In [ ]:
!pip install numpy==1.26.4
!pip install pandas==1.5.3
!pip install scikit-learn==1.3.2
!pip install joblib==1.3.2
!pip install matplotlib==3.8.2
!pip install seaborn==0.13.1
!pip install plotly==5.18.0
!pip install shap==0.42.1


In [ ]:
# retrain_fixed.py
"""
Robust training script that:
 - Drops identifier/time columns (vm_id, timestamp) by default
 - Imputes missing values for numeric/categorical features
 - Uses OneHotEncoder in a version-safe way
 - Avoids multiprocessing (no n_jobs=-1 or parallel CV)
 - Falls back to single-fit evaluation if CV fails
 - Saves: model.pkl, scaler.pkl, label_encoder.pkl
"""
import pandas as pd
import joblib
import inspect
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# ----------------------------
# 1. Load data
# ----------------------------
df = pd.read_csv("small.csv")   # adjust path if needed
print("Data shape:", df.shape)

# ----------------------------
# 2. Drop identifier/timecolumns (common in VM logs)
# ----------------------------
drop_cols = []
for c in ["vm_id", "timestamp"]:
    if c in df.columns:
        drop_cols.append(c)
if drop_cols:
    print("Dropping identifier/time columns:", drop_cols)
    df = df.drop(columns=drop_cols)

# ----------------------------
# 3. Prepare X, y
# ----------------------------
# assume last column is the target (as before)
X = df.iloc[:, :-1].copy()
y = df.iloc[:, -1].copy()

print("\nMissing per column:\n", X.isna().sum())
print("\nTarget distribution:\n", y.value_counts())

# ----------------------------
# 4. Label encode target
# ----------------------------
label_enc = LabelEncoder()
y_enc = label_enc.fit_transform(y)

# ----------------------------
# 5. Identify categorical and numeric columns
# ----------------------------
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print("\nCategorical cols:", cat_cols)
print("Numeric cols:", num_cols)

# ----------------------------
# 6. Safe OneHotEncoder for different sklearn versions
# ----------------------------
ohe_init_sig = inspect.signature(OneHotEncoder.__init__)
ohe_kwargs = {"handle_unknown": "ignore"}
if "sparse_output" in ohe_init_sig.parameters:
    ohe_kwargs["sparse_output"] = False
elif "sparse" in ohe_init_sig.parameters:
    ohe_kwargs["sparse"] = False

ohe = OneHotEncoder(**ohe_kwargs)

# ----------------------------
# 7. Imputers & scalers
# ----------------------------
num_imputer = SimpleImputer(strategy="median")     # robust to outliers
cat_imputer = SimpleImputer(strategy="most_frequent", fill_value="__MISSING__")
scaler_for_pipeline = StandardScaler()

# Fit a standalone scaler for numeric columns (to save scaler.pkl)
if len(num_cols) > 0:
    # fit on full numeric data after imputing median to avoid NaNs
    num_data = X[num_cols].copy()
    num_data_imputed = num_imputer.fit_transform(num_data)
    fitted_scaler = StandardScaler().fit(num_data_imputed)
else:
    fitted_scaler = None

# For pipeline preprocessor create fresh instances (they will be fitted inside the pipeline)
num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]) if num_cols else "drop"
cat_pipeline = Pipeline([("imputer", SimpleImputer(strategy="most_frequent", fill_value="__MISSING__")), ("ohe", ohe)]) if cat_cols else "drop"

transformers = []
if cat_cols:
    transformers.append(("cat", cat_pipeline, cat_cols))
if num_cols:
    transformers.append(("num", num_pipeline, num_cols))

preprocessor = ColumnTransformer(transformers=transformers, remainder="drop")

# ----------------------------
# 8. Models (no heavy parallelism)
# ----------------------------
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    # avoid n_jobs=-1 to prevent multiprocessing issues in constrained envs
    "RandomForest": RandomForestClassifier(n_jobs=1, random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_jobs=1, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(probability=True)
}

# ----------------------------
# 9. Train-test split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.20, random_state=42, stratify=y_enc
)

# ----------------------------
# 10. Evaluate with CV (safe: n_jobs=1) with fallback handling
# ----------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}
for name, estimator in models.items():
    pipe = Pipeline([("preprocessor", preprocessor), ("model", estimator)])
    print(f"\nEvaluating {name} ...")
    try:
        # Use cross_val_score with n_jobs=1 to avoid multiprocessing issues
        scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="f1_weighted", n_jobs=1)
        results[name] = {"cv_f1_mean": float(scores.mean()), "cv_f1_std": float(scores.std())}
        print(f"  CV F1 (weighted): {scores.mean():.4f} +/- {scores.std():.4f}")
    except Exception as e:
        # If CV fails, attempt a simple fit on train and evaluate on test (safe single-run)
        print(f"  CV failed for {name}: {e}")
        try:
            pipe.fit(X_train, y_train)
            preds = pipe.predict(X_test)
            acc = accuracy_score(y_test, preds)
            f1 = f1_score(y_test, preds, average="weighted")
            results[name] = {"cv_f1_mean": float(f1), "cv_f1_std": 0.0, "note": "fallback_single_eval"}
            print(f"  Fallback evaluation -> acc: {acc:.4f}, f1 (weighted): {f1:.4f}")
        except Exception as e2:
            results[name] = {"error": str(e2)}
            print(f"  Fallback also failed for {name}: {e2}")

# ----------------------------
# 11. Select best model
# ----------------------------
valid = {k: v for k, v in results.items() if "cv_f1_mean" in v}
if not valid:
    raise RuntimeError("No model evaluated successfully. See printed errors above.")

best_name = max(valid, key=lambda k: valid[k]["cv_f1_mean"])
print("\nBest model selected:", best_name, valid[best_name])
best_estimator = models[best_name]

# ----------------------------
# 12. Retrain best model on FULL DATA
# ----------------------------
final_pipeline = Pipeline([("preprocessor", preprocessor), ("model", best_estimator)])
print("\nFitting final pipeline on full dataset ...")
final_pipeline.fit(X, y_enc)

# ----------------------------
# 13. Evaluate final model on holdout (X_test)
# ----------------------------
try:
    preds_test = final_pipeline.predict(X_test)
    print("\nFinal model test accuracy:", accuracy_score(y_test, preds_test))
    print("Final model test F1 (weighted):", f1_score(y_test, preds_test, average="weighted"))
    print("Classification Report:\n", classification_report(y_test, preds_test, target_names=label_enc.classes_))
except Exception as e:
    print("Could not evaluate final model on X_test:", e)

# ----------------------------
# 14. Save artifacts
# ----------------------------
joblib.dump(final_pipeline, "model.pkl")
joblib.dump(fitted_scaler, "scaler.pkl")
joblib.dump(label_enc, "label_encoder.pkl")
print("\nSaved artifacts: model.pkl, scaler.pkl, label_encoder.pkl")
